In [1]:
%config IPCompleter.greedy=True
from notebook.services.config import ConfigManager
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})

{'CodeCell': {'cm_config': {'autoCloseBrackets': False}},
 'Cell': {'cm_config': {'lineNumbers': True}}}

In [2]:
from plnn.mnist_sequential import Net
import torch
import torch.nn as nn
from torchvision.datasets.mnist import MNIST
import torch.utils.data
import numpy as np
import torch.nn.functional as F
from torchvision import datasets, transforms

In [3]:
def generate_domain(input_tensor, eps_size):
    return torch.stack((input_tensor - eps_size, input_tensor + eps_size))

In [4]:
model = Net()
model.load_state_dict(torch.load('save/mnist_sequential_cnn.pt'))
model.cuda()
dataset = MNIST('./data', train=True, download=True,
                transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,), (0.3081,))
                ])),  # load the testing dataset
batch_size=10
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=False, pin_memory=True)
# test_loader = torch.utils.data.DataLoader(dataset, batch_size=1)#retrieve items 1 at a time
seed = 0

In [5]:
class VerificationNetwork(nn.Module):
    def __init__(self, in_features, out_features, base_network, out_function, true_class_index):
        super(VerificationNetwork, self).__init__()
        self.true_class_index = true_class_index
        self.out_function = out_function
        self.base_network = base_network
        self.out_features = out_features
        self.in_features = in_features
        self.property_layer = self.attach_property_layers(self.base_network, self.true_class_index)

    '''need to  repeat this method for each class so that it describes the distance between the corresponding class 
    and the closest other class'''
    def attach_property_layers(self, model: Net, true_class_index: int):
        n_classes = model.fc2.out_features
        cases = []
        for i in range(n_classes):
            if i == true_class_index:
                continue
            case = [0] * n_classes  # list of zeroes
            case[true_class_index] = 1  # sets the property to 1
            case[i] = -1
            cases.append(case)
        weights = np.array(cases)
        weightTensor = nn.Linear(in_features=n_classes, out_features=n_classes,
                                 bias=False)
        weightTensor.weight.data = torch.from_numpy(weights).float()
        return weightTensor

    def forward(self, x):
        x = self.base_network(x)
        x = self.property_layer(x)
        print(x)
        print(x.size())
        return torch.min(x,dim=1,keepdim=True)


In [6]:
#get the data and label
data, target =next(iter(test_loader))
print(f'data size:{data.size()}')
# print(data[0])
# create the domain
domain_raw = generate_domain(data,0.001)
data_size=data.size()
print(f'domain size:{domain_raw.size()}')


data size:torch.Size([10, 1, 28, 28])
domain size:torch.Size([2, 10, 1, 28, 28])


In [7]:
test_out=model(data.cuda())
print(test_out.size())

torch.Size([10, 10])


In [8]:
domain=domain_raw.view(2,batch_size,-1)
print(domain.size())

torch.Size([2, 10, 784])


In [9]:
# global_ub_point, global_ub = net.get_upper_bound(domain)
# global_lb = net.get_lower_bound(domain)


def get_upper_bound(domain):
    #we try get_upper_bound
    nb_samples = 1024
    nb_inp = domain.size()[2:]  #get last dimensions
    print(nb_inp)
    # Not a great way of sampling but this will be good enough
    # We want to get rows that are >= 0
    rand_samples_size = [batch_size, nb_samples] + list(nb_inp)
    print(rand_samples_size)
    rand_samples = torch.zeros(rand_samples_size)
    print(rand_samples.size())
    # print(rand_samples)
    rand_samples.uniform_(0, 1)
    # print(rand_samples)
    print(rand_samples.size())
    domain_lb = domain.select(0, 0).contiguous()
    domain_ub = domain.select(0, 1).contiguous()
    # print(domain_lb)
    print(domain_lb.size())
    # print(domain_ub)
    print(domain_ub.size())
    domain_width = domain_ub - domain_lb
    print(domain_width.size())
    print(domain_lb.view([batch_size, 1] + list(nb_inp)).size())
    print(domain_width.view([batch_size, 1] + list(nb_inp)).size())
    domain_lb = domain_lb.view([batch_size, 1] + list(nb_inp)).expand(
        [batch_size, nb_samples] + list(nb_inp))  #expand the initial point for the number of examples
    print(domain_lb.size())
    domain_width = domain_width.view([batch_size, 1] + list(nb_inp)).expand(
        [batch_size, nb_samples] + list(nb_inp))  #expand the width for the number of examples
    print(domain_width.size())
    #those should be the same
    print(domain_width.size())
    print(rand_samples.size())
    inps = domain_lb + domain_width * rand_samples
    # print(inps) #each row shuld be different
    print(inps.size())
    #now flatten the first dimension into the second
    flattened_size = [inps.size(0) * inps.size(1)] + list(inps.size()[2:])
    print(flattened_size)
    #rearrange the tensor so that is consumable by the model
    print(data_size)
    examples_data_size = [flattened_size[0]] + list(data_size[1:])  #the expected dimension of the example tensor
    print(examples_data_size)
    var_inps = torch.Tensor(inps).view(examples_data_size)
    print(var_inps.size())  #should match data_size
    print(inps.size())
    # print(inps[0][0])
    # print(inps[0][1])
    # print(var_inps[0][0])
    # print(var_inps[1][0])
    outs = model.forward(var_inps.cuda())  #gets the input for the values
    print(outs.size())
    print(outs[0])  #those two should be very similar but different because they belong to two different random examples
    print(outs[1])
    print(target.unsqueeze(1))
    target_expanded = target.unsqueeze(1).expand(
        [batch_size, nb_samples])  #generates nb_samples copies of the target vector, all rows should be the same
    print(target_expanded.size())
    print(target_expanded)
    target_idxs = target_expanded.contiguous().view(
        batch_size * nb_samples)  #contains a list of indices that tells which columns out of the 10 classes to pick
    print(target_idxs.size())  #the first dimension should match
    print(outs.size())
    print(outs[target_idxs[0]].size())
    outs_true_class = outs.gather(1, target_idxs.cuda().view(-1,
                                                             1))  #we choose dimension 1 because it's the one we want to reduce
    print(outs_true_class.size())
    # print(outs[0])
    # print(target_idxs[1])
    # print(outs[1][0])#these two should be similar but different because they belong to different examples
    # print(outs[0][0])
    print(outs_true_class.size())
    outs_true_class_resized = outs_true_class.view(batch_size, nb_samples)
    print(outs_true_class_resized.size())  #resize outputs so that they each row is a different element of each batch
    upper_bound, idx = torch.min(outs_true_class_resized,
                                 dim=1)  #this returns the distance of the network output from the given class, it selects the class which is furthest from the current one
    print(upper_bound.size())
    print(idx.size())
    print(idx)
    print(upper_bound)
    # rearranged_idx=idx.view(list(inps.size()[0:2]))
    # print(rearranged_idx.size()) #rearranged idx contains the indexes of the minimum class for each example, for each element of the batch
    print(f'idx size {idx.size()}')
    print(f'inps size {inps.size()}')
    print(idx[0])
    # upper_bound = upper_bound[0]
    unsqueezed_idx = idx.cuda().view(-1, 1)
    print(f'single size {inps[0][unsqueezed_idx[0][0]][:].size()}')
    print(f'single size {inps[1][unsqueezed_idx[1][0]][:].size()}')
    print(f'single size {inps[2][unsqueezed_idx[2][0]][:].size()}')
    ub_point = [inps[x][idx[x]][:].numpy() for x in range(idx.size()[0])]
    ub_point = torch.tensor(ub_point)
    print(
        ub_point)  #ub_point represents the input that amongst all examples returns the minimum response for the appropriate class
    print(ub_point.size())
    # print(unsqueezed_idx.size())
    # ub_point = torch.gather(inps.cuda(),1,unsqueezed_idx.cuda())#todo for some reason it doesn't want to work
    # print(ub_point.size())
    return ub_point, upper_bound


In [11]:
#now try to do the lower bound
import gurobipy as grb

In [93]:
'''
input_domain: Tensor containing in each row the lower and upper bound
              for the corresponding dimension
'''
lower_bounds = []
upper_bounds = []
gurobi_vars = []
# These three are nested lists. Each of their elements will itself be a
# list of the neurons after a layer.

gurobi_model = grb.Model()
gurobi_model.setParam('OutputFlag', False)
gurobi_model.setParam('Threads', 1)

In [94]:
input_domain=domain.select(1,0)#we use a single domain, not ready for parallelisation yet
print(input_domain.size())

torch.Size([2, 784])


In [95]:
## Do the input layer, which is a special case
inp_lb = []
inp_ub = []
inp_gurobi_vars = []
# for dim in range(input_domain.size()[1]):
dim=0
ub=input_domain[1][dim]#check this value, it can be messed up
lb=input_domain[0][dim]
print(f'ub={ub} lb={lb}')
assert ub>lb , "ub should be greater that lb"
#     print(f'ub={ub} lb={lb}')
v = gurobi_model.addVar(lb=lb, ub=ub, obj=0,
                      vtype=grb.GRB.CONTINUOUS,
                      name=f'inp_{dim}')
inp_gurobi_vars.append(v)
inp_lb.append(lb)
inp_ub.append(ub)
gurobi_model.update()

ub=-0.4232129752635956 lb=-0.42521294951438904


In [96]:
lower_bounds.append(inp_lb)
upper_bounds.append(inp_ub)
gurobi_vars.append(inp_gurobi_vars)

In [97]:
print(lower_bounds[0][0])
print(upper_bounds[0][0])

tensor(-0.4252)
tensor(-0.4232)


In [98]:
layer_idx = 1
# for layer in model.layers:
layer = model.layers[0]
new_layer_lb = []
new_layer_ub = []
new_layer_gurobi_vars = []
neuron_idx=0
ub = layer.bias.data[neuron_idx]
lb = layer.bias.data[neuron_idx]
print(f'bias_ub={ub} bias_lb={lb}')
lin_expr = layer.bias.data[neuron_idx].item() #adds the bias to the linear expression
prev_neuron_idx = 0
coeff = layer.weight.data[neuron_idx, prev_neuron_idx]#picks the weight between the two neurons
print(f'coeff={coeff} upper={coeff*upper_bounds[-1][prev_neuron_idx]} lower={coeff*lower_bounds[-1][prev_neuron_idx]}')
assert coeff*lower_bounds[-1][prev_neuron_idx]!=coeff*upper_bounds[-1][prev_neuron_idx]
ub =ub+ coeff*upper_bounds[-1][prev_neuron_idx]#multiplies the ub
lb =ub+ coeff*lower_bounds[-1][prev_neuron_idx]#multiplies the lb
print(f'ub={ub} lb={lb}')
assert ub!=lb
lin_expr += coeff.item() * gurobi_vars[-1][prev_neuron_idx]#multiplies the unknown by the coefficient
print(lin_expr)

bias_ub=6.959481716156006 bias_lb=6.959481716156006
coeff=0.015848033130168915 upper=-0.006707093212753534 lower=-0.006738788913935423
ub=6.952774524688721 lb=6.946035861968994
<gurobi.LinExpr: 6.959481716156006 + 0.015848033130168915 inp_0>


In [99]:
v = gurobi_model.addVar(lb=lb, ub=ub, obj=0,
                              vtype=grb.GRB.CONTINUOUS,
                              name=f'lay{layer_idx}_{neuron_idx}')
gurobi_model.addConstr(v == lin_expr)
gurobi_model.update()
print(f'v={v}')

v=<gurobi.Var lay1_0>


In [100]:
gurobi_model.setObjective(v, grb.GRB.MINIMIZE)
gurobi_model.optimize()
print(f'gurobi status {gurobi_model.status}')

gurobi status 2


In [22]:
## Do the other layers, computing for each of the neuron, its upper
## bound and lower bound
layer_idx = 1
# for layer in model.layers:
layer = model.layers[0]
print( type(layer))
new_layer_lb = []
new_layer_ub = []
new_layer_gurobi_vars = []
if type(layer) is nn.Linear:
    for neuron_idx in range(layer.weight.size(0)):
        ub = layer.bias.data[neuron_idx]
        lb = layer.bias.data[neuron_idx]
        lin_expr = layer.bias.data[neuron_idx].item()
        for prev_neuron_idx in range(layer.weight.size(1)):
            coeff = layer.weight.data[neuron_idx, prev_neuron_idx]
            if coeff >= 0:
                ub += coeff*upper_bounds[-1][prev_neuron_idx]
                lb += coeff*lower_bounds[-1][prev_neuron_idx]
            else:
                ub += coeff*lower_bounds[-1][prev_neuron_idx]
                lb += coeff*upper_bounds[-1][prev_neuron_idx]
            lin_expr += coeff.item() * gurobi_vars[-1][prev_neuron_idx]
        v = gurobi_model.addVar(lb=lb, ub=ub, obj=0,
                              vtype=grb.GRB.CONTINUOUS,
                              name=f'lay{layer_idx}_{neuron_idx}')
        print(f'ub={ub} lb={lb}')
#         assert ub>lb, "ub is not bigger than lb"
#         print(f'v={v} lin_expr={lin_expr}')
        gurobi_model.addConstr(v == lin_expr)
        gurobi_model.update()

        gurobi_model.setObjective(v, grb.GRB.MINIMIZE)
        gurobi_model.optimize()
        print(f'gurobi status {gurobi_model.status}')
        assert gurobi_model.status == 2, "LP wasn't optimally solved"
        # We have computed a lower bound
        lb = v.X
        v.lb = lb

        # Let's now compute an upper bound
        gurobi_model.setObjective(v, grb.GRB.MAXIMIZE)
        gurobi_model.update()
        gurobi_model.reset()
        gurobi_model.optimize()
        assert gurobi_model.status == 2, "LP wasn't optimally solved"
        ub = v.X
        v.ub = ub

        new_layer_lb.append(lb)
        new_layer_ub.append(ub)
        new_layer_gurobi_vars.append(v)

<class 'torch.nn.modules.linear.Linear'>
ub=7.0670485496521 lb=7.0670485496521
gurobi status 3


AssertionError: LP wasn't optimally solved